## Importing Libraries

In [43]:
import os.path 
import numpy as np
import pandas as pd

_Playing around with pathnames to get the hang of os.path functions_

In [44]:
credit_path = os.path.expanduser("~/Danny/Administration/Finance/Haushaltsbudget/abrechnungen/credit")
debit_path = os.path.expanduser("~/Danny/Administration/Finance/Haushaltsbudget/abrechnungen/debit")

jan_cc_path = os.path.join(credit_path, "544271XXXXXX1411_2022-01-01_2022-01-31-2.csv")
feb_cc_path = os.path.join(credit_path, "544271XXXXXX1411_2022-02-01_2022-02-28.csv")
mar_cc_path = os.path.join(credit_path, "544271XXXXXX1411_2022-03-01_2022-03-31.csv")
apr_cc_path = os.path.join(credit_path, "544271XXXXXX1411_2022-04-01_2022-04-30.csv")

In [45]:
dc_files = os.listdir(debit_path)
jan_dc_file = [f for f in dc_files if "01-01" in f][0]
jan_dc_path = os.path.join(debit_path, jan_dc_file)

feb_dc_file = [f for f in dc_files if "02-01" in f][0]
feb_dc_path = os.path.join(debit_path, feb_dc_file)

mar_dc_file = [f for f in dc_files if "03-01" in f][0]
mar_dc_path = os.path.join(debit_path, mar_dc_file)

apr_dc_file = [f for f in dc_files if "04-01" in f][0]
apr_dc_path = os.path.join(debit_path, apr_dc_file)

## Defining Helpful Functions

#### ...to import my bank statements (credit card and current account) and prepare them in a pd.DataFrame

In [46]:
def read_statement(path_to_file):
    """
    Reads in a csv_file with specific format (sep = ";") and returns a cleaned list ready for pandas DataFrame
    """

    with open(path_to_file, encoding = 'utf_16') as f:
        lines = f.readlines()
        f.close()

    newLines = []
    for i in range(len(lines)):
        line = lines[i]
        line = line.strip('\n')
#         line = line.replace(',', '.') # This is not good. If I have an amount larger than 1k, it will not parse
# Try doing the , to . conversion within the pd.DataFrame already
        line = line.split(';')
        newLines.append(line)

    return (newLines)

In [47]:
def clean_df(lines):
    """
    Returns a pd.DataFrame object from cleaned lines of bank statement
    """

    nan_value = float("NaN")
    data = pd.DataFrame(data = lines[1:], columns = lines[0])
    data["Betrag"].replace('[.]', '', regex=True, inplace=True)
    data["Betrag"].replace('[,]', '.', regex=True, inplace=True)
    data.replace("", nan_value, inplace=True)
    data.dropna(how='all', axis=1, inplace=True)
    data["Betrag"] = data["Betrag"].astype(float)
    data["Buchungs-Info"] = data["Buchungs-Info"].astype(str) # This is necessary so we don't have NaN as floats in this row, because we later work with this row assuming it contains only strings
    data["Notiz"] = data["Notiz"].astype(str) # This is necessary so we don't have NaN as floats in this row, because we later work with this row assuming it contains only strings
    return data

In [62]:
def filter_entries(data):
    data = data[~data.Notiz.apply(lambda s: "ignore" in s.lower())] # Remove entries flagged as ignored
    data = data[~data.Notiz.apply(lambda s: "gehalt" in s.lower())] # Remove my salary
    data = data[~data["Buchungs-Info"].apply(lambda s: "ihre zahlung vormonat" in s.lower())] # Remove cc payment
    data = data[~data["Buchungs-Info"].apply(lambda s: "kreditkartenrechnung" in s.lower())] # Remove cc payment

## In some cases I receive money back for shared or borrowed expenses. 
## In most cases I want to ignore those, but sometimes it's part of an expense that needs to stay visible
## Then I don't want to ignore all positive values... Need to find a better (more fine-grained) solution here.

    
    data = data[~((data.Betrag >= 0) & (data.Notiz.apply(lambda s: s == "nan")))] # Remove all positive values, since I'm tracking spending only here
    
    return data

**Importing credit and debit card statements**

- Read Lines from raw CSV
- Clean lines and import into a data frame with appropriate data types in columns for filtering later
- Logically filtering out entries (like removing ignored or positive entries)

In [49]:
lines_jan_cc = read_statement(jan_cc_path)
jan_cc = clean_df(lines_jan_cc)
jan_cc = filter_entries(jan_cc)

lines_jan_dc = read_statement(jan_dc_path)
jan_dc = clean_df(lines_jan_dc)
jan_dc = filter_entries(jan_dc)

In [50]:
lines_feb_cc = read_statement(feb_cc_path)
feb_cc = clean_df(lines_feb_cc)
feb_cc = filter_entries(feb_cc)

lines_feb_dc = read_statement(feb_dc_path)
feb_dc = clean_df(lines_feb_dc)
feb_dc = filter_entries(feb_dc)

In [52]:
lines_mar_cc = read_statement(mar_cc_path)
mar_cc = clean_df(lines_mar_cc)
mar_cc = filter_entries(mar_cc)

lines_mar_dc = read_statement(mar_dc_path)
mar_dc = clean_df(lines_mar_dc)
mar_dc = filter_entries(mar_dc)

In [53]:
mar_cc[((mar_cc.Betrag >= 0 ) & (mar_cc.Notiz.apply(lambda s: s != "nan")))]

,Valutadatum,Buchungsdatum,Partner Kontonummer,Bankleitzahl,Betrag,Währung,Buchungs-Info,Buchungsreferenz,Notiz
39,11.03.2022,14.03.2022,40005195600,20111,24.55,EUR,REV*roksh.com,201112203112ALV-004127892957,Hofer
54,07.03.2022,08.03.2022,NaN,NaN,408.00,EUR,Zuzahlung s Kreditkarte,201112203072ALV-141655481079,Ipad


In [63]:
lines_apr_cc = read_statement(apr_cc_path)
apr_cc = clean_df(lines_apr_cc)
apr_cc = filter_entries(apr_cc)

lines_apr_dc = read_statement(apr_dc_path)
apr_dc = clean_df(lines_apr_dc)
apr_dc = filter_entries(apr_dc)

In [67]:
apr_cc

,Valutadatum,Buchungsdatum,Partner Kontonummer,Bankleitzahl,Betrag,Währung,Buchungs-Info,Buchungsreferenz,Notiz
0,28.04.2022,29.04.2022,40005195600,20111,-3.60,EUR,Das Kafa,201112204282ALV-000901550689,nan
1,28.04.2022,29.04.2022,40005195600,20111,-33.29,EUR,Mjam Gmbh_llxp-hh07,201112204282ALV-000900027734,nan
2,28.04.2022,29.04.2022,40005195600,20111,-9.80,EUR,NOODLE KING,201112204282ALV-000708363998,nan
3,27.04.2022,28.04.2022,40005195600,20111,-8.14,EUR,Hofer Dankt,201112204272ALV-234944448191,nan
4,27.04.2022,28.04.2022,40005195600,20111,-1.95,EUR,BIPA dankt,201112204272ALV-234749096149,nan
5,26.04.2022,27.04.2022,40005195600,20111,-16.81,EUR,SPAR DANKT 4958,201112204262ALV-235259313998,nan
6,26.04.2022,27.04.2022,40005195600,20111,-10.10,EUR,APOTHEKE Z ERZENGEL MI,201112204262ALV-235104873625,nan
7,26.04.2022,27.04.2022,40005195600,20111,-3.65,EUR,APOTHEKE Z ERZENGEL MI,201112204262ALV-235104871853,nan
8,21.04.2022,22.04.2022,40005195600,20111,-0.91,EUR,Hofer Dankt,201112204212ALV-000239648588,nan
9,21.04.2022,22.04.2022,40005195600,20111,-7.00,EUR,O'CHICKEN NICE,201112204212ALV-000000048656,nice-food


**Defining categories which I have in by budget**

This doesn't happen yet because, what I'm actually searching for in my statements are **items**, which I have in sub-categories.

If I want to have the higher level categories, I still need to connect the categorised items to their respective categories

In [56]:
categories = {"Living Costs": ["Rent", "Laundry", "Household"], 
              "Transport": ["Jahreskarte", "Parking", "Scooter", "Uber", "ShareNow", "Public Transport"],
              "Food": ["Groceries", "Eating Out", "Breakfast", "Lunch"],
              "Leisure": ["Coffee", "Alcohol", "Activity"],
              "Health/Beauty": ["Hygiene", "Lenses", "Hair", "Pharma"],
              "Subscriptions": ["Spotify", "Phone"],
              "Shopping": ["Clothes", "Electronics"],
              "Sports": ["Tennis", "Sports"],
              "Contingency": ["Uncategorised", "Admin"]
            }

In [57]:
items = {
    "Rent": ["Miete"],
    "Household": ["Household"],
    "Laundry": ["Laundry, ARWAG"],
    "Groceries": ["Groceries", "BILLA", "SPAR", "Hofer", "Penny"],
    "Hair": ["Hair"],
    "Hygiene": ["Hygiene", "BIPA"],
    "Lenses": ["Lenses", "VOLENS"],
    "Pharma": ["Apotheke", "Pharma"],
    "Eating Out": ["Eating Out", "Pumpui", "Aumann", "McDonalds", "Mjam", "PIZZERIA"], #Need Note here
    "Breakfast": ["Breakfast", "Der Mann", "NOEBAUER", "Ströck"], #Need Note here
    "Lunch": ["Lunch", "UNI-ECK", "NINO", "Noodle King", "Swingkitchen"], #Need Note here
    "Scooter": ["Scooter", "TIER", "LIM"],
    "Uber": ["Uber", "UBER"],
    "ShareNow": ["ShareNow", "SN329"],
    "Sports": ["Sports"],
    "Spotify": ["Spotify Fam"],
    "Phone": ["XOXO"],
    "Public Transport": ["OEBB", ],
    "Alcohol": ["alcohol", "NEEDLE", "PICKWICKS", ], #Need Note here
    "Activity": ["Activity", "NTRY"],  #Need Note here
    "Uncategorised": ["Uncategorised"],
    "Admin": ["Admin"],
    "Tennis": ["Tennis"],
    "Laundry": ["Laundry", "ARWAG"],
    "Coffee": ["Coffee", "Kafa", ],
    "Shopping": ["Shopping", "electronics", "gifts", "clothes"],
    "Contingency": ["ipad"],
    "vacations": ["nice-food"]
        }

**Defining function, which takes a string (Buchungs-Info or Notiz) and checks whether that string contains some item name**

In [58]:
def is_item_category(s, item_category):
    candidates = items[item_category]
    for candidate in candidates:
        if candidate.lower() in s.lower():
            return True
    return False

In [59]:
def notiz_is_category(s, item_category):
    candidates = categories.values()
    for candidate in candidates:
        if candidate.lower() in s.lower():
            return True
    return False

### Defining the function which does the categorising

In [60]:
def evaluate_statement(data, month):

    categorised_data = pd.DataFrame(columns = ["item", "amount"])
    
    print()
    print ('{} expenses:'.format(month))
    tot_sum = 0
    n_items = 0
    tot_items = len(data)
    remaining_items = data
    
#     for candidates in categories.values():
#         for item in candidates:

#             if len(found_entries) == 0: continue

#             sum_item_category = round(found_entries["Betrag"].sum(), 2)
#             print(item, ": ", sum_item_category)
#             tot_sum += sum_item_category
#             n_items += len(found_entries)
#             remaining_items = remaining_items[~remaining_items["Notiz"].apply(lambda s: item in s)]

#             new_line = pd.DataFrame({"item": item, "amount" : -sum_item_category}, index = [0])
#             categorised_data = pd.concat([categorised_data, new_line],  ignore_index=True)
    
    for item_category in items.keys():
        try:
            found_entries_items = remaining_items[remaining_items["Buchungs-Info"].apply(lambda x: is_item_category(x, item_category))]
            remaining_items = remaining_items[~remaining_items["Buchungs-Info"].apply(lambda x: is_item_category(x, item_category))]
            found_entries_notiz = remaining_items[remaining_items["Notiz"].apply(lambda s: is_item_category(s, item_category))]
            remaining_items = remaining_items[~remaining_items["Notiz"].apply(lambda s: is_item_category(s, item_category))]
        except KeyError:
            break
        if len(found_entries_items) == 0 and len(found_entries_notiz) == 0: continue

        found_entries = pd.concat([found_entries_items, found_entries_notiz]).drop_duplicates()
        
        sum_item_category = round(found_entries["Betrag"].sum(), 2)
        print(item_category, ": ", sum_item_category)
        tot_sum += sum_item_category
        n_items += len(found_entries)
        remaining_items = remaining_items[~remaining_items["Buchungs-Info"].apply(lambda x: is_item_category(x, item_category))]

        if item_category in categorised_data.item:
            categorised_data[categorised_data.item == item_category]["amount"] = categorised_data[categorised_data.item == item_category]["amount"] + sum_item_category
        else:
            new_line = pd.DataFrame({"item": item_category, "amount" : -sum_item_category}, index = [0])
            categorised_data = pd.concat([categorised_data, new_line], ignore_index=True)
        
    print("-------------------------------------")
    print ("Categorised {} items from {}".format(n_items, tot_items))
    print ("Total: ", round(tot_sum, 2))

    print("-------------------------------------")
    print ("Uncategorised Items: ", tot_items - n_items, " with total of: ", round(data["Betrag"].sum() - tot_sum, 2), "\n")
    if len(remaining_items) > 0 :
        print (remaining_items[["Buchungsdatum", "Betrag", "Buchungs-Info", "Notiz"]])
    
    return categorised_data

In [57]:
jan_data = pd.concat([jan_cc, jan_dc], ignore_index=True)

jan_data_eval = evaluate_statement(jan_data, "January")


January expenses:
Groceries :  -74.41
Eating Out :  -119.9
Breakfast :  -16.5
Lunch :  -94.5
Scooter :  -14.07
Uber :  -10.23
Spotify :  -2.5
Phone :  -13.22
Public Transport :  -5.0
Alcohol :  -57.0
Activity :  -49.76
Uncategorised :  -84.5
-------------------------------------
Categorised 41 items from 41
Total:  -541.59
-------------------------------------
Uncategorised Items:  0  with total of:  0.0 



In [58]:
jan_data_eval.to_csv("jan_data_evaluated.csv")

In [219]:
feb_data = pd.concat([feb_cc, feb_dc])

feb_data_eval = evaluate_statement(feb_data, "Febuary")


Febuary expenses:
Rent :  -825.0
Household :  -12.35
Laundry :  -4.5
Groceries :  -189.28
Hygiene :  -51.7
Eating Out :  -68.42
Breakfast :  -5.35
Lunch :  -48.4
Scooter :  -4.75
Uber :  -21.8
ShareNow :  -10.06
Spotify :  -2.5
Phone :  -9.97
Alcohol :  -192.0
Activity :  -60.0
Admin :  -20.7
Tennis :  -130.0
Coffee :  -17.65
-------------------------------------
Categorised 78 items from 80
Total:  -1674.43
-------------------------------------
Uncategorised Items:  2  with total of:  864.59 

   Buchungsdatum  Betrag            Buchungs-Info                  Notiz
14    23.02.2022  556.62  Zuzahlung s Kreditkarte  Geld für MacBook Kauf
2     24.02.2022  307.97        Sent from Revolut           Nice Freddie


In [220]:
feb_data_eval.to_csv("feb_data_evaluated.csv")

In [223]:
mar_data = pd.concat([mar_cc, mar_dc])

mar_data_eval = evaluate_statement(mar_data, "March")


March expenses:
Rent :  -625.0
Laundry :  -5.5
Groceries :  -277.84
Hygiene :  -4.99
Lenses :  -64.89
Pharma :  -10.1
Eating Out :  -19.0
Breakfast :  -7.9
Lunch :  -114.9
Scooter :  -13.93
ShareNow :  -16.77
Sports :  -32.86
Spotify :  -2.5
Phone :  -9.98
Alcohol :  -40.5
Activity :  -80.0
Tennis :  -171.43
Coffee :  -14.3
Shopping :  -83.23
Contingency :  -507.98
-------------------------------------
Categorised 76 items from 76
Total:  -2103.6
-------------------------------------
Uncategorised Items:  0  with total of:  0.0 



In [224]:
mar_data_eval.to_csv("mar_data_evaluated.csv")

In [65]:
apr_data = pd.concat([apr_cc, apr_dc])

apr_data_eval = evaluate_statement(apr_data, "April")


April expenses:
Rent :  -625.0
Laundry :  -3.0
Groceries :  -202.99
Hair :  -32.0
Hygiene :  -1.95
Pharma :  -13.75
Eating Out :  -33.29
Breakfast :  -6.2
Lunch :  -75.4
Scooter :  -4.55
ShareNow :  -17.2
Spotify :  -2.5
Phone :  -9.98
Public Transport :  -3.8
Alcohol :  -20.0
Coffee :  -3.6
Shopping :  -40.34
vacations :  -401.65
-------------------------------------
Categorised 62 items from 62
Total:  -1497.2
-------------------------------------
Uncategorised Items:  0  with total of:  -0.0 



In [66]:
apr_data_eval.to_csv("apr_data_evaluated.csv")

In [471]:
# First check Notiz for category name
# Then check Buchungs-Info for items which can lead to category via items and categories dicts
# Finally (via gui, eventually) print entries where no category was found and ask for category
# If no category is given, put in uncategorised 

# For some entries (where I paid for others or when I get money back), I will either have to ignore them or
# Consolidate them. In particular, if I get money back from several people put it all together and create
# Effective entries of the money I actually spent. Since this is still a review process, I need to find a GUI way
# To implement this

For some entries (where I paid for others or when I get money back), I will either have to ignore them or
Consolidate them. In particular, if I get money back from several people put it all together and create
Effective entries of the money I actually spent. Since this is still a review process, I need to find a GUI way
To implement this

My thought is that for every entry that doesn't have a category I either set a category or I can remove/create a new entry.